In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x116a98fd0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x114dda588>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x114dda6d8>)

# 构建一个多层卷积网络

In [2]:
#权重初始化
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)


In [3]:
#卷积和池化
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [4]:
#占位符
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [5]:
#第一层卷积
W_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])
W_conv1,b_conv1

(<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(32,) dtype=float32_ref>)

In [6]:
x_image=tf.reshape(x,[-1,28,28,1])
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [7]:
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)
h_conv1,h_pool1

(<tf.Tensor 'Relu:0' shape=(?, 28, 28, 32) dtype=float32>,
 <tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>)

In [8]:
#第二层卷积
W_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])
W_conv2,b_conv2

(<tf.Variable 'Variable_2:0' shape=(5, 5, 32, 64) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>)

In [9]:
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)
h_conv2,h_pool2

(<tf.Tensor 'Relu_1:0' shape=(?, 14, 14, 64) dtype=float32>,
 <tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>)

In [10]:
#全连接层
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])
W_fc1,b_fc1

(<tf.Variable 'Variable_4:0' shape=(3136, 1024) dtype=float32_ref>,
 <tf.Variable 'Variable_5:0' shape=(1024,) dtype=float32_ref>)

In [11]:
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_pool2_flat,h_fc1

(<tf.Tensor 'Reshape_1:0' shape=(?, 3136) dtype=float32>,
 <tf.Tensor 'Relu_2:0' shape=(?, 1024) dtype=float32>)

In [12]:
#Dropout
keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)
h_fc1_drop

<tf.Tensor 'dropout/mul:0' shape=(?, 1024) dtype=float32>

In [13]:
#输出层
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
W_fc2,b_fc2,y_conv

(<tf.Variable 'Variable_6:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'Variable_7:0' shape=(10,) dtype=float32_ref>,
 <tf.Tensor 'Softmax:0' shape=(?, 10) dtype=float32>)

In [14]:
#训练及评估
cross_entropy=-tf.reduce_sum(y_*tf.log(y_conv))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
for i in range(6000):
    batch_xs,batch_ys=mnist.train.next_batch(50)
    if i%100==0:
        train_accuracy=accuracy.eval(session=sess,feed_dict={
            x:batch_xs,y_:batch_ys,keep_prob:1.0
        })
        print('step %d,training accuracy %g' % (i,train_accuracy))
    train_step.run(session=sess,feed_dict={x:batch_xs,y_:batch_ys,keep_prob:1.0})

step 0,training accuracy 0.98
step 100,training accuracy 1
step 200,training accuracy 1
step 300,training accuracy 1
step 400,training accuracy 0.98
step 500,training accuracy 1
step 600,training accuracy 1
step 700,training accuracy 1
step 800,training accuracy 1
step 900,training accuracy 1
step 1000,training accuracy 1
step 1100,training accuracy 1
step 1200,training accuracy 1
step 1300,training accuracy 0.96
step 1400,training accuracy 1
step 1500,training accuracy 1
step 1600,training accuracy 0.98
step 1700,training accuracy 0.98
step 1800,training accuracy 1
step 1900,training accuracy 1
step 2000,training accuracy 1
step 2100,training accuracy 1
step 2200,training accuracy 1
step 2300,training accuracy 1
step 2400,training accuracy 1
step 2500,training accuracy 1
step 2600,training accuracy 1
step 2700,training accuracy 1
step 2800,training accuracy 1
step 2900,training accuracy 1
step 3000,training accuracy 1
step 3100,training accuracy 1
step 3200,training accuracy 0.98
step

In [16]:
print('test accuracy %g' % accuracy.eval(session=sess,feed_dict={
    x:mnist.test.images,
    y_:mnist.test.labels,
    keep_prob:1.0
}))

test accuracy 0.9861
